In [207]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import jpholiday

In [208]:
def load_price_data(file_path):
    df = pd.read_csv(file_path, encoding="shift-jis")

    df['受渡日'] = pd.to_datetime(df['受渡日'], format='%Y/%m/%d')

    def time_from_code(code):
        hour = (code - 1) // 2
        minute = 30 * ((code - 1) % 2)
        return f"{hour:02d}:{minute:02d}"

    df['time'] = df['時刻コード'].apply(time_from_code)
    df['datetime'] = pd.to_datetime(df['受渡日'].astype(str) + ' ' + df['time'])
    df['price'] = df['エリアプライス北海道(円/kWh)']

    return df[['datetime', 'price']]

In [209]:
# --- 3. 任意の2年分データを読み込む ---
df_input1 = load_price_data('./notebook/spot_summary_2015.csv')
df_input2 = load_price_data('./notebook/spot_summary_2022.csv')
df_all = pd.concat([df_input1, df_input2], ignore_index=True)

In [210]:
#月、時刻、日付、曜日（0:月〜6:日）、祝日判定を追加。

# 土日または祝日を 'holiday'、それ以外を 'weekday' として day_type に分類。

df_all['month'] = df_all['datetime'].dt.month
df_all['time'] = df_all['datetime'].dt.time
df_all['date'] = df_all['datetime'].dt.date
df_all['weekday'] = df_all['datetime'].dt.weekday
df_all['is_holiday'] = df_all['date'].apply(jpholiday.is_holiday)
df_all['day_type'] = df_all.apply(
    lambda row: 'holiday' if row['weekday'] >= 5 or row['is_holiday'] else 'weekday', axis=1
)


In [211]:
# 価格が0.01円の行を True に。
df_all['is_001'] = df_all['price'] == 0.01

In [212]:
df_all.head(10)

,datetime,price,month,time,date,weekday,is_holiday,day_type,is_001
0,2015-04-01 00:00:00,10.11,4,00:00:00,2015-04-01,2,False,weekday,False
1,2015-04-01 00:30:00,10.11,4,00:30:00,2015-04-01,2,False,weekday,False
2,2015-04-01 01:00:00,10.11,4,01:00:00,2015-04-01,2,False,weekday,False
3,2015-04-01 01:30:00,10.11,4,01:30:00,2015-04-01,2,False,weekday,False
4,2015-04-01 02:00:00,10.56,4,02:00:00,2015-04-01,2,False,weekday,False
5,2015-04-01 02:30:00,10.56,4,02:30:00,2015-04-01,2,False,weekday,False
6,2015-04-01 03:00:00,10.56,4,03:00:00,2015-04-01,2,False,weekday,False
7,2015-04-01 03:30:00,10.91,4,03:30:00,2015-04-01,2,False,weekday,False
8,2015-04-01 04:00:00,10.26,4,04:00:00,2015-04-01,2,False,weekday,False
9,2015-04-01 04:30:00,10.13,4,04:30:00,2015-04-01,2,False,weekday,False


In [213]:
# 月×曜日タイプ×時間単位の0.01円出現頻度
# 0.01円が出現した時間帯を月×曜日タイプ×時刻でグルーピング。
# 出現頻度順にソート。
grouped = (
    df_all[df_all['is_001']]
    .groupby(['month', 'day_type', 'time'])
    .size()
    .reset_index(name='count')
    .sort_values(['month', 'count', 'time'], ascending=[True, False, True])
)

In [214]:
# 月別の0.01円出現回数（平均化）
# 月別の0.01円出現回数をカウントし、2年分あるので2で割って平均化。
monthly_001 = df_all.groupby('month')['is_001'].sum().reset_index(name='count_001')
monthly_001['avg_count'] = (monthly_001['count_001'] / 2).astype(int)

In [215]:
monthly_001.head(10)

,month,count_001,avg_count
0,1,0,0
1,2,13,6
2,3,150,75
3,4,170,85
4,5,203,101
5,6,67,33
6,7,44,22
7,8,104,52
8,9,58,29
9,10,43,21


In [216]:
# 月別・時間別の平均価格
# 月ごと・時間帯ごとの平均価格を算出。
monthly_avg = df_all.groupby(['month', 'time'])['price'].mean().reset_index()
monthly_avg.head(10)

,month,time,price
0,1,00:00:00,14.259677
1,1,00:30:00,13.928710
2,1,01:00:00,13.677258
3,1,01:30:00,13.686935
4,1,02:00:00,13.537742
5,1,02:30:00,13.561774
6,1,03:00:00,13.643065
7,1,03:30:00,13.517097
8,1,04:00:00,13.691452
9,1,04:30:00,13.766452


In [217]:
# --- 標準モデルのベースを作成 ---
standard_dates = pd.date_range('2023-01-01', '2023-12-31', freq='D')
standard_rows = []
for date in standard_dates:
    month = date.month
    weekday = date.weekday()
    is_holiday = jpholiday.is_holiday(date)
    day_type = 'holiday' if weekday >= 5 or is_holiday else 'weekday'
    for _, row in monthly_avg[monthly_avg['month'] == month].iterrows():
        standard_rows.append({
            'datetime': datetime.combine(date, row['time']),
            'month': month,
            'time': row['time'],
            'day_type': day_type,
            'price': row['price']
        })
df_standard = pd.DataFrame(standard_rows)

In [218]:
# 0.01円ブロック割り当て
# 各月で：
# 平均出現数を block_size で割ってブロック数決定。
# 対象月の日付をその数に分割（np.array_split）。
# 頻度の高い時間帯から、ブロックごとに「連続3コマ」を挿入。
# df_standard の該当インデックスの price を 0.01 に変更。
# ブロックごとに 連続性を持たせて、かつ分散 させる工夫。

# --- 月ごとのブロック分割と連続0.01円割り当て ---
block_size = 5  # 0.01円を連続3コマにする
for month in range(1, 13):
    avg_count = monthly_001.loc[monthly_001['month'] == month, 'avg_count'].values[0]
    num_blocks = avg_count // block_size

    if num_blocks == 0:
        continue  # この月には0.01円を挿入しない

    # 対象月の日付一覧を取得してブロック分割
    month_dates = sorted(df_standard[df_standard['month'] == month]['datetime'].dt.date.unique())
    blocks = np.array_split(month_dates, num_blocks)

    top_times = grouped[grouped['month'] == month]
    selected_indices = []

    for i, block in enumerate(blocks):
        block_days = set(block)
        block_df = df_standard[
            (df_standard['month'] == month) &
            (df_standard['datetime'].dt.date.isin(block_days))
        ].copy()

        for _, row in top_times.iterrows():
            sub = block_df[
                (block_df['day_type'] == row['day_type']) &
                (block_df['time'] == row['time']) &
                (~block_df.index.isin(selected_indices))
            ]
            if not sub.empty:
                center_idx = sub.index[0]
                all_times = df_standard[df_standard['month'] == month]
                center_loc = all_times.index.get_loc(center_idx)
                for offset in range(-(block_size // 2), block_size // 2 + 1):
                    idx = center_loc + offset
                    if 0 <= idx < len(all_times):
                        selected_indices.append(all_times.index[idx])
                break  # 次のブロックへ

    # 0.01円割当
    df_standard.loc[selected_indices, 'price'] = 0.01

In [219]:
# --- 出力 ---
df_standard.sort_values('datetime', inplace=True)
df_standard.reset_index(drop=True, inplace=True)
df_standard[['datetime', 'price']].to_csv('jepx_standard_model_seasonal_blocked.csv', index=False)
print("標準モデルを出力しました（0.01円コマ分散＆連続性あり）")

標準モデルを出力しました（0.01円コマ分散＆連続性あり）
